In [6]:
import pandas as pd

# Load the datasets from the provided CSV files but only the first 3000 rows for ease of calculation
return_df = pd.read_csv('return.csv', nrows=3000)
accruals_compustat_df = pd.read_csv('accurals_compustat.csv', nrows=3000)

# Creating a shifted dataframe to calculate the changes in values for each company
accruals_compustat_shifted = accruals_compustat_df.groupby('cusip').shift(-1)

# Calculating changes in the necessary fields
accruals_compustat_df['change_in_act'] = accruals_compustat_shifted['act'] - accruals_compustat_df['act']
accruals_compustat_df['change_in_lct'] = accruals_compustat_shifted['lct'] - accruals_compustat_df['lct']
accruals_compustat_df['average_total_assets'] = (accruals_compustat_shifted['at'] + accruals_compustat_df['at']) / 2

#'che' is cash and equivalent and 'dlc' and 'txp' need to be excluded from the current liabilities
accruals_compustat_df['change_in_current_liabilities'] = accruals_compustat_df['change_in_lct'] - (accruals_compustat_shifted['dlc'] - accruals_compustat_df['dlc']) - (accruals_compustat_shifted['txp'] - accruals_compustat_df['txp'])

# The change in non-cash current assets is the change in 'act' minus the change in 'che'
accruals_compustat_df['change_in_noncash_current_assets'] = accruals_compustat_df['change_in_act'] - (accruals_compustat_shifted['che'] - accruals_compustat_df['che'])

# Calculating the accruals based on the provided formula
accruals_compustat_df['accruals'] = accruals_compustat_df['change_in_noncash_current_assets'] - accruals_compustat_df['change_in_current_liabilities']
accruals_compustat_df['accruals'] /= accruals_compustat_df['average_total_assets']

# Dropping the rows with NaN values as they will not be useful for comparison and sorting
accruals_compustat_df_cleaned = accruals_compustat_df.dropna(subset=['accruals'])

# Sorting the stocks by accruals in reverse order
sorted_accruals = accruals_compustat_df_cleaned.sort_values(by='accruals', ascending=False)

# Displaying the sorted dataframe
sorted_accruals[['cusip', 'accruals']]


,cusip,accruals
1679,98159J200,9.925287
1688,98159J200,3.966387
2870,007531106,2.161074
2310,004930202,1.440442
1635,003870300,1.114139
...,...,...
1676,98159J200,-5.000000
1680,98159J200,-6.411538
1675,98159J200,-20.533333
1678,98159J200,-48.800000
